In [1]:
import os
import sys

root = os.path.abspath(os.path.join('..', '..'))
test_dir = os.path.join(root, 'backend', 'docking', 'tests')

In [ ]:
# We can use rdkit to generate small molecule conformers necessary
# for Rosetta ligand docking:

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import rdMolAlign

# Take example output from this directory:
example_hits = '2gs6_picks.sdf'
max_confs = 10

supplier = Chem.SDMolSupplier(example_hits)

# To align generated conformers to the input ligand's pose, we need to:
# 1. Store the original coordinates.
# 2. After generating conformers, align each conformer to the original pose.
for i, mol in enumerate(supplier):
    if mol is None:
        continue

    # Add hydrogens to the input hit ligand:
    # (does this step shift the coordinates?)
    mol = Chem.AddHs(mol, addCoords=True)
    lig_name = f'2gs6_hit_{i+1}'

    # Store original coordinates as a reference conformer
    ref_conf_id = mol.id # I don't think this is specifying the original pose correctly.

    # Generate conformers
    conf_ids = AllChem.EmbedMultipleConfs(mol, numConfs=max_confs,
                                          useExpTorsionAnglePrefs=True,
                                          useBasicKnowledge=True)
    
    # Align each conformer to the original pose (doesn't seem to work):
    for conf_id in conf_ids:
        rdMolAlign.AlignMol(mol, mol, prbCid=conf_id, refCid=ref_conf_id)

    mol_conf_file = os.path.join(test_dir, f'{lig_name}_conformers.sdf')
    with Chem.SDWriter(mol_conf_file) as writer:
        for conf_id in conf_ids:
            writer.write(mol, confId=conf_id)


In [ ]:
# Now we use Rosetta's molfile_to_params.py script to generate
# a parameters file for each ligand in addition to a pdb file
# for the hit ligand and another for each of its conformers.

# Note, this will only work if running with an installed Rosetta version.

import subprocess
import glob

conf_files = sorted(glob.glob(os.path.join(test_dir, '2gs6_hit_*_conformers.sdf')))

script_path = os.path.join(test_dir, '..', 'molfile_to_params.py')
script_path = os.path.abspath(script_path)

for i, file in enumerate(conf_files):
    lig_pdb_tag = f'NM{i+1}'
    prefix = os.path.basename(os.path.splitext(file)[0])
    cmd = [
        sys.executable, script_path,
        '-n', lig_pdb_tag,
        '-p', prefix,
        '--chain=X',
        '--conformers-in-one-file', file
    ]
    subprocess.run(cmd, check=True, cwd=test_dir)

In [11]:
# Concatenate the prepared ligand pdb and its receptor:

for i in range(1, 13):
    rec_pdb_file = '2gs6_receptor.pdb'
    lig_pdb_file = os.path.join(test_dir, f'2gs6_hit_{i}.pdb')
    complex_pdb_file = os.path.join(test_dir, f'2gs6_hit_{i}_complex.pdb')

    with open(rec_pdb_file, 'r') as f_rec, open(lig_pdb_file, 'r') as f_lig, open(complex_pdb_file, 'w') as f_out:
        # Write receptor atoms first:
        for line in f_rec:
            if line.startswith('ATOM'):
                f_out.write(line)
        f_out.write('TER\n')

        # Write ligand atoms second:
        for line in f_lig:
            if line.startswith('HETATM'):
                f_out.write(line)
        f_out.write('END\n')
